# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [23]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, accuracy_score, fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('always')

[nltk_data] Downloading package punkt to /home/vanducng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vanducng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vanducng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vanducng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table("disaster_response",engine)
X = df["message"]
Y = df.iloc[:,4:]

In [25]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [26]:
def tokenize(text):
    # Normalize
    normalized_text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize
    words = word_tokenize(normalized_text)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    clean_words = [lemmatizer.lemmatize(w, pos="n").strip() for w in words]
    clean_words = [lemmatizer.lemmatize(w, pos="v").strip() for w in clean_words]
    
    return clean_words
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f314c4c7cb0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def report(model, X_test, y_test):
    """
    Generate classification report
    
    Parameters:
        model: built model
        X_test: X_test dataset
        y_test: y_test dataset
        
    Returns:
        Printing classification report
    """
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [8]:
report(pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.37      0.06      0.10      1249
           1       0.76      0.97      0.85      3954
           2       0.00      0.00      0.00        39

    accuracy                           0.74      5242
   macro avg       0.38      0.34      0.32      5242
weighted avg       0.66      0.74      0.67      5242

request
              precision    recall  f1-score   support

           0       0.84      0.98      0.90      4359
           1       0.38      0.06      0.10       883

    accuracy                           0.83      5242
   macro avg       0.61      0.52      0.50      5242
weighted avg       0.76      0.83      0.77      5242

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        24

    accuracy                           1.00      5242
   macro avg       0.50      0.50      0.50      524

              precision    recall  f1-score   support

           0       0.75      0.95      0.84      3786
           1       0.56      0.17      0.27      1456

    accuracy                           0.73      5242
   macro avg       0.66      0.56      0.55      5242
weighted avg       0.70      0.73      0.68      5242

floods
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4794
           1       0.11      0.00      0.01       448

    accuracy                           0.91      5242
   macro avg       0.51      0.50      0.48      5242
weighted avg       0.85      0.91      0.87      5242

storm
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4735
           1       0.49      0.04      0.07       507

    accuracy                           0.90      5242
   macro avg       0.70      0.52      0.51      5242
weighted avg       0.87      0.90      0.86      5242

fire
 

/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f3158e96170>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f3158e96170>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [10]:
parameters = {
    "clf__estimator__n_estimators": [50, 100],
    "clf__estimator__min_samples_split": [2, 4]
}
cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, y_train)
report(cv, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.37      0.06      0.10      1249
           1       0.76      0.97      0.85      3954
           2       0.00      0.00      0.00        39

    accuracy                           0.74      5242
   macro avg       0.38      0.34      0.32      5242
weighted avg       0.66      0.74      0.67      5242

request
              precision    recall  f1-score   support

           0       0.84      0.98      0.91      4359
           1       0.44      0.06      0.11       883

    accuracy                           0.83      5242
   macro avg       0.64      0.52      0.51      5242
weighted avg       0.77      0.83      0.77      5242

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        24

    accuracy                           1.00      5242
   macro avg       0.50      0.50      0.50      524

              precision    recall  f1-score   support

           0       0.75      0.95      0.84      3786
           1       0.58      0.18      0.27      1456

    accuracy                           0.74      5242
   macro avg       0.66      0.56      0.56      5242
weighted avg       0.70      0.74      0.68      5242

floods
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4794
           1       0.14      0.00      0.01       448

    accuracy                           0.91      5242
   macro avg       0.53      0.50      0.48      5242
weighted avg       0.85      0.91      0.87      5242

storm
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      4735
           1       0.48      0.03      0.06       507

    accuracy                           0.90      5242
   macro avg       0.70      0.51      0.50      5242
weighted avg       0.87      0.90      0.86      5242

fire
 

/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [20]:
cv.best_params_

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
pipeline2 = Pipeline([
    ("vect", CountVectorizer(tokenize)),
    ("best", TruncatedSVD()),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(AdaBoostClassifier()))
])
pipeline2.fit(X_train, y_train)
report(pipeline2, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1249
           1       0.75      1.00      0.86      3954
           2       0.00      0.00      0.00        39

    accuracy                           0.75      5242
   macro avg       0.25      0.33      0.29      5242
weighted avg       0.57      0.75      0.65      5242

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4359
           1       0.00      0.00      0.00       883

    accuracy                           0.83      5242
   macro avg       0.42      0.50      0.45      5242
weighted avg       0.69      0.83      0.76      5242

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        24

    accuracy                           1.00      5242
   macro avg       0.50      0.50      0.50      524

/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5225
           1       0.00      0.00      0.00        17

    accuracy                           1.00      5242
   macro avg       0.50      0.50      0.50      5242
weighted avg       0.99      1.00      1.00      5242

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5178
           1       0.00      0.00      0.00        64

    accuracy                           0.99      5242
   macro avg       0.49      0.50      0.50      5242
weighted avg       0.98      0.99      0.98      5242

other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      5000
           1       0.00      0.00      0.00       242

    accuracy                           0.95      5242
   macro avg       0.48      0.50      0.49      5242
weighted avg       0.91      0.95      0.

/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [21]:
# Param tunning
parameters = {
    "tfidf__use_idf": (True, False), 
    "clf__estimator__n_estimators": [50, 100],
    "clf__estimator__learning_rate": [1,2] 
}
cv2 = GridSearchCV(pipeline2, param_grid=parameters)
cv2.fit(X_train, y_train)
report(cv2, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1249
           1       0.75      1.00      0.86      3954
           2       0.00      0.00      0.00        39

    accuracy                           0.75      5242
   macro avg       0.25      0.33      0.29      5242
weighted avg       0.57      0.75      0.65      5242

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      4359
           1       0.00      0.00      0.00       883

    accuracy                           0.83      5242
   macro avg       0.42      0.50      0.45      5242
weighted avg       0.69      0.83      0.76      5242

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        24

    accuracy                           1.00      5242
   macro avg       0.50      0.50      0.50      524

/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       0.72      1.00      0.84      3786
           1       0.40      0.00      0.01      1456

    accuracy                           0.72      5242
   macro avg       0.56      0.50      0.42      5242
weighted avg       0.63      0.72      0.61      5242

floods
              precision    recall  f1-score   support

           0       0.91      1.00      0.96      4794
           1       0.33      0.00      0.00       448

    accuracy                           0.91      5242
   macro avg       0.62      0.50      0.48      5242
weighted avg       0.86      0.91      0.87      5242

storm
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      4735
           1       0.00      0.00      0.00       507

    accuracy                           0.90      5242
   macro avg       0.45      0.50      0.47      5242
weighted avg       0.82      0.90      0.86      5242

fire
 

/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vanducng/.local/share/virtualenvs/vanducng.dev-hNwUqbaf/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

### 9. Export your model as a pickle file

In [31]:
with open('model.pkl', 'wb') as f:
    pickle.dump(pipeline2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [37]:
top_categories = df.iloc[:, 4:].sum().sort_values(ascending=False)[1:11]

In [45]:
top_cat_names = top_categories.index.to_list()

In [44]:
genres = df.groupby("genre").sum()[top_categories.index.to_list()]

In [50]:
genres.index.to_list()

['direct', 'news', 'social']

In [53]:
genres["request"]

genre
direct    3671
news       602
social     207
Name: request, dtype: int64